In [188]:
import random
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#from scipy.stats import norm



In [ ]:
numcards = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
opcards1 = ['+', '-', '/', '*', '']
opcards2 = ['+', '-', '/', '*']

Primelist = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
Fibolist = [2, 3, 5, 8, 13, 21, 34, 55, 89]
SqubeList = [1, 2, 4, 8, 9, 16, 25, 27, 36, 49, 64, 81, 100]

Triggered = {"Prime": 0, "Fibo":0, "Sqube" : 0}
Scores = {"Prime":[],"Fibo":[],"Sqube":[]}

prevanswer = 0

In [190]:


def numextract(str):
    
    exp=str
    
    for symbol in opcards2:
        exp = exp.replace(symbol, ',')
    
    return exp.split(',')



In [ ]:
def exprgen(numlimit):
    answer = 0
    while answer<1 or answer>100:   #critical point: cap range; right now its 1 to 100 inclusive
        op = '0'
        expr = ''

        for i in range(numlimit):
            num = random.choice(numcards)
            expr += num
                
            if i < (numlimit-1):
                if num == '10' or op == '':
                    op = random.choice(opcards2)
                else:
                    op = random.choice(opcards1)    
                expr += op
        answer = int(eval(expr)) 
    return expr


In [192]:

def primes(val, numberlist):
    
    primemult = 0
    if val in Primelist:
        Triggered['Prime'] += 1
        primemult = sum(1 for num in numberlist if num in Primelist)
        Scores['Prime'].append(val*primemult)


In [ ]:
def fibos (preval, val, numberlist):
    fibotrigger = sum(1 for num in numberlist if num in Fibolist)
    
    if fibotrigger>1:     # critical point : how many fibos required to trigger; right now its one.
        Triggered ['Fibo'] += 1
        Scores['Fibo'].append(val + preval)

In [194]:
def squbes (val, numlist):
    digsum = 0
    for num in numlist:
      if num in SqubeList:
        digsum += num//10 + num%10 
    
    if digsum > 0:
        Triggered['Sqube'] += 1
        Scores['Sqube'].append(val+digsum)

In [ ]:


for i in range (100):      #critical point: the number of sims to run     #also, run collective simualtions or five turns each with preval resetting at start of each turn
    expr = (exprgen(4))    #critical point: the numbers in the exoression generate: right now 4   (should do them for 3 aswell)
    answer = int(eval(expr))
    nums = [int(x) for x in numextract(expr)]
    fibos(prevanswer,answer, nums)
    primes(answer, nums)
    squbes(answer, nums)
    prevanswer = answer

print(Scores)     
print(Triggered)

#have to collect the data in pandas and analyze
#have to plot curves using numpy, matplotlib


{'Prime': [21, 9, 0, 83, 14, 7, 21, 6, 4, 11, 2, 38, 37, 10, 0, 17, 6, 159, 10, 2, 97, 11, 10, 62, 111, 3, 9, 7, 39, 6, 14, 44, 10, 7, 17, 7, 67, 13, 33, 57], 'Fibo': [52, 48, 74, 133, 83, 61, 13, 87, 36, 106, 17, 87, 63, 56, 58, 41, 57, 40, 16, 36, 68, 45, 27, 29, 15, 108, 116, 82, 25, 93, 48, 102, 99, 18, 15, 24, 62, 56, 21, 50, 23, 93, 81], 'Sqube': [16, 12, 24, 52, 41, 62, 98, 16, 72, 57, 22, 9, 89, 11, 21, 24, 53, 17, 11, 12, 105, 32, 41, 19, 24, 80, 30, 72, 12, 55, 24, 11, 35, 34, 29, 61, 60, 19, 15, 40, 46, 19, 18, 33, 58, 18, 23, 28, 14, 55, 79, 21, 17, 99, 50, 14, 109, 17, 13, 23, 16, 16, 23, 12, 31, 19, 77, 56, 25, 25, 24, 29, 21, 88, 35, 28]}
{'Prime': 40, 'Fibo': 43, 'Sqube': 76}
